## Load data for all pyramidal cells from all trials into a dataframe of ephys objects

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from session_utils import *
from ephys import *
from ephys_utils import select_spikes_by_trial

lfp_sampling_rate = 1000

# Load all session names and paths as dict
session_list = find_all_sessions(sheet_path = 'https://docs.google.com/spreadsheets/d/1_Xs5i-rHNTywV-WuQ8-TZliSjTxQCCqGWOD2AL_LIq0/edit#gid=0',
                                 data_path = '/home/isabella/Documents/isabella/jake/recording_data',
                                 sorting_suffix = 'sorting_ks2_custom')

# Initialise DataFrame and explicitly set dtype for 'clusters_inc' to 'object'
df_all_cells = pd.DataFrame(data = None, index = session_list.keys(), columns=['clusters_inc'], dtype='object')


for i, session_path in enumerate(session_list.values()):
    session = list(session_list.keys())[i]
    
    # Get IDs of included clusters from postprocessing.Select Clusters All Sessions.ipynb
    clusters_inc = np.load(f'{session_path}/clusters_inc.npy', allow_pickle = True)
    
    if len(clusters_inc) > 0:
        df_all_cells.at[session, 'clusters_inc'] = clusters_inc

        # Create ephys object for session and add to dataframe
        obj = ephys(recording_type = 'nexus', path = session_path)

        # Load spikes for all included clusters
        obj.load_spikes(clusters_inc)
        
        # Reorganise spikes by trial, starting at t=0 at each trial
        obj.spikes_by_trial = select_spikes_by_trial(obj.spike_data, obj.trial_iterators, obj.trial_offsets)

        # Load position data for all trials
        obj.load_pos(obj.trial_iterators, output_flag = False, reload_flag = False)

        # Get unique channels with included cells
        obj.good_channels = np.unique(obj.spike_data['cluster_info'].loc(axis=1)['ch'])
        
        # # Load LFP for all trials for channels with units
        # obj.load_lfp(obj.trial_iterators, lfp_sampling_rate, channels = obj.good_channels, reload_flag = True)

        # Add data to frame
        df_all_cells.at[list(session_list.keys())[i], 'ephys_object'] = obj

    print(f'Loaded session {session}')

# Pickle dataframe for loading elsewhere
df_all_cells.to_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/df_all_cells.pkl')
print(f'All data saved to /home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/df_all_cells.pkl')

df_all_cells.head()

In [ ]:
import numpy as np
import pandas as pd
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from position_analysis import *
from get_theta_frequencies import get_theta_frequencies
from get_theta_phase import get_spike_theta_phase
from ephys import *
from postprocessing.spatial_analysis import speed_filter_spikes
from rayleigh_vector import rayleigh_vector


## Calculate theta phase for all pyramidal cells

In [ ]:
# Load pickled data
df_all_cells = pd.read_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/df_all_cells.pkl')

lfp_sampling_rate = 1000

# Drop rows with no included clusters
df_all_cells = df_all_cells.dropna()
print(f'{len(df_all_cells.index)} sessions retained')

# Loop through all sessions
for index, obj in df_all_cells['ephys_object'].items():
    
    # # Find t-maze and open field trials for session
    # t_maze_trials = [i for i, s in enumerate(obj.trial_list) if 't-maze' in s]
    # open_field_trials = [i for i, s in enumerate(obj.trial_list) if 'open-field' in s]
    
    # Initialise output variables
    obj.cluster_phases = [{} for _ in range(len(obj.trial_list))]
    obj.cluster_sectors = [{} for _ in range(len(obj.trial_list))]
    obj.cluster_rayleigh = [{} for _ in range(len(obj.trial_list))]
    
    # Load LFP if not already loaded
    obj.load_lfp(obj.trial_iterators, lfp_sampling_rate, channels = obj.good_channels, reload_flag = False)

    # Load LFP and position for each channel, get theta phase and position sector for each spike
    for i in obj.trial_iterators:
        
        # Extract XY position data for trial
        trial_pos = obj.pos_data[i]['xy_position']

        # Convert to DataFrame and rename columns to match original channel
        lfp_df = pd.DataFrame(obj.lfp_data[i]['data'])
        lfp_df.columns = obj.good_channels

        # Find peak theta frequencies for each channel and make dict of {channel: theta_freq}
        theta_freqs = get_theta_frequencies(obj.lfp_data[i]['data'], lfp_sampling_rate)
        theta_freqs = dict(zip(obj.good_channels, theta_freqs))


        # Loop through each good cluster
        for cluster in obj.spike_data['cluster_info'].index:
            # Get channel for cluster
            channel = obj.spike_data['cluster_info'].loc[cluster, 'ch']

            # Get peak theta frequency
            theta_freq = theta_freqs[channel]

            ## SPIKE PHASE
            # Extract LFP data for the recording channel
            lfp_data_for_channel = lfp_df.loc[:, channel]

            # Extract spike times for the cluster
            cluster_spike_times = obj.spikes_by_trial[i]['spike_times'][obj.spikes_by_trial[i]['spike_clusters'] == cluster]
            
            # Filter for speed (function from spatial_analysis.py)
            spike_times_filtered = speed_filter_spikes(current_trial_spikes = {cluster: cluster_spike_times}, #Needs to be dict, key not important
                                                        speed_data = obj.pos_data[i]['speed'],
                                                        position_sampling_rate = obj.pos_data[i]['pos_sampling_rate'],
                                                        speed_lower_bound = 10, #2.5 cm/s
                                                        speed_upper_bound = 400) #100 cm/s

            # Compute theta phases for the spike times
            spike_phases = get_spike_theta_phase(lfp_data_for_channel, spike_times_filtered[cluster], lfp_sampling_rate, theta_freq)

            # Store in the dictionary
            obj.cluster_phases[i][cluster] = spike_phases    

            ## SPIKE POSITION SECTOR
            # Assign spike times to nearest position sector
            spike_pos = [[], []]
            for t in spike_times_filtered[cluster]:
                spike_pos_index = round(t * 50)/50
                spike_pos[0].append(trial_pos[spike_pos_index].values[0])
                spike_pos[1].append(trial_pos[spike_pos_index].values[1])

            # spike_pos is now an array with the structure  [[x1, x2, x3 ...], [y1, y2, y3 ...]] for each spike
            # Get position sector for each spike
            spike_sectors = assign_sectors(pd.DataFrame(spike_pos), pos_header = obj.pos_data[i]['header'])

            # add sectors to dict
            obj.cluster_sectors[i][cluster] = spike_sectors

            ## RAYLEIGH VECTOR
            # Compute rayleigh vector for each cluster
            obj.cluster_rayleigh[i][cluster] = rayleigh_vector(spike_phases)
    
    
    # Re-assign object to dataframe
    df_all_cells.loc[index, 'ephys_object'] = obj
    print(f'Session {index} processed')

# Pickle dataframe for loading elsewhere
df_all_cells.to_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/df_all_cells_theta.pkl')

## Plot theta phase vs position sector for t-maze and theta phase for open field

In [ ]:
# Plotting functions
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import scipy.stats

# Function to get phases for a given sector within a specific cluster
def get_phases_for_sector_in_cluster(cluster_sectors, cluster_phases, sector, cluster_id):
    indices = np.where(cluster_sectors[int(cluster_id)] == sector)[0]
    return cluster_phases[int(cluster_id)][indices]

# Function to plot polar plots for a specific cluster
def plot_theta_phase_by_position_cluster(obj, trial_iterator, cluster_id, session):
    
    # Define the directory where you want to save the figure
    save_directory = f'/home/isabella/Documents/isabella/jake/ephys_analysis/figures/P{obj.age}_{session}'
    
    # Check if the directory exists, if not, create it
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
        
    ### For T-MAZE TRIALS:
    if 't-maze' in obj.trial_list[trial_iterator]:
        unique_sectors = range(1,13)
        fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(15, 10), subplot_kw={'projection': 'polar'})

        for idx, sector in enumerate(unique_sectors):
            row_idx = idx // 4
            col_idx = idx % 4
            sector = int(sector)

            phases = get_phases_for_sector_in_cluster(obj.cluster_sectors[trial_iterator], obj.cluster_phases[trial_iterator], sector, cluster_id)

            # Remove any NaN/ infinite values
            phases = phases[np.isfinite(phases)]

            # Calculate circular mean of theta phase
            mean_phase = scipy.stats.circmean(phases, nan_policy = 'omit')
            std_phase = scipy.stats.circstd(phases, nan_policy = 'omit')

            # Histogram of the phases
            n, bins, patches = axes[row_idx, col_idx].hist(phases, bins=30, alpha=0.6)

            # Add a red line indicating the mean phase
            axes[row_idx, col_idx].axvline(mean_phase, color='r', linestyle='--', linewidth=2)

            # Add a sector indicating standard deviation
            # axes[row_idx, col_idx].axvspan(mean_phase-std_phase, mean_phase+std_phase, color='r', alpha = 0.4)

            # Set the title for the current subplot
            axes[row_idx, col_idx].set_title(f"Sector {sector}")

        plt.suptitle(f'Theta phase vs position sector for cluster {cluster_id} from trial {obj.trial_list[trial_iterator]}', fontsize = 25)
        plt.tight_layout()
        plt.savefig(f'{save_directory}/{cluster_id}_trial-{trial_iterator}_theta_phase_by_sector.png')
        plt.show()
        
    ### FOR OPEN FIELD TRIALS
    if 'open-field' in obj.trial_list[trial_iterator]:
        
        # Get all phases for cluster
        phases = obj.cluster_phases[trial_iterator][cluster_id]
        
        # Remove any NaN/ infinite values
        phases = phases[np.isfinite(phases)]   
        
        # Calculate circular mean of theta phase
        mean_phase = scipy.stats.circmean(phases, nan_policy = 'omit')
        std_phase = scipy.stats.circstd(phases, nan_policy = 'omit')
        
        # Create plot
        fig, axes = plt.subplots(figsize = (5, 5), subplot_kw={'projection': 'polar'})
        
        # Histogram of the phases
        n, bins, patches = axes.hist(phases, bins=30, alpha=0.6)

        # Add a red line indicating the mean phase
        axes.axvline(mean_phase, color='r', linestyle='--', linewidth=2)
        
        plt.suptitle(f'Theta phase distribution for cluster {cluster_id}\n from trial {obj.trial_list[trial_iterator]}', fontsize = 15)
        plt.tight_layout()
        plt.savefig(f'{save_directory}/{cluster_id}_trial-{trial_iterator}_theta_phase_by_sector.png')
        plt.show()
    
#### Plot theta phase vs position

# Loop through all sessions and make all plots per cluster
for session, obj in df_all_cells['ephys_object'].items():
    
    # Loop through each good cluster
    for cluster in obj.spike_data['cluster_info'].index:
        
        # Loop through each trial and plot 12-sector plot for t-maze single polar plot for open field
        for trial in obj.trial_iterators:
            plot_theta_phase_by_position_cluster(obj, trial, cluster, session)

## Plot rate maps for pyramidal cells

In [ ]:
import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from postprocessing.spatial_analysis import make_rate_maps, plot_cluster_across_sessions, speed_filter_spikes
import pandas as pd
import matplotlib.pyplot as plt
from ephys_utils import select_spikes_by_trial, transform_spike_data
from spatial_information import spatial_info


# Load pickled data
df_all_cells = pd.read_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/df_all_cells.pkl') if df_all_cells is None else df_all_cells
# Drop rows with no included clusters
df_all_cells = df_all_cells.dropna()
print(f'{len(df_all_cells.index)} sessions retained')

spatial_info_all = pd.DataFrame(columns = ['bits_per_spike', 'bits_per_sec', 'age'], dtype = 'object')
rates_all = pd.DataFrame(columns = ['mean_rates', 'max_rates', 'age'], dtype = 'object')
rate_maps_all = pd.DataFrame(columns = ['open-field_1', 'open-field_2', 't-maze_1', 't-maze_2', 'age'], dtype = 'object')
occupancy_maps_all = pd.DataFrame(columns = ['open-field_1', 'open-field_2', 't-maze_1', 't-maze_2', 'age'], dtype = 'object')

# Calculate rate maps
for session, obj in df_all_cells['ephys_object'].items():
    
    # Loop through trials and generate rate maps
    rate_maps = {}
    raw_rate_maps = {}
    smoothed_pos_maps = {}
    raw_pos_maps = {}
    max_rates = {}
    mean_rates = {}
    bits_per_spike = {}
    bits_per_sec = {}
    
    # Make rate maps for all trials
    for trial, trial_name in enumerate(obj.trial_list):
        
#         # Load unloaded position data if any
#         obj.load_pos(trial, reload_flag = False)

        # Select spikes for current trial and transform to create a dict of {cluster: spike_times}
        current_trial_spikes = select_spikes_by_trial(obj.spike_data, trial, obj.trial_offsets)
        current_trial_spikes = transform_spike_data(current_trial_spikes)[trial]
        
        # Filter spikes for speed
        current_trial_spikes_filtered = speed_filter_spikes(current_trial_spikes,
                                                            speed_data = obj.pos_data[trial]['speed'],
                                                            position_sampling_rate = obj.pos_data[trial]['pos_sampling_rate'],
                                                            speed_lower_bound = 2.5, #2.5 cm/s
                                                            speed_upper_bound = 100) #100 cm/s
        
        # Calculate rate maps
        rate_maps[trial], raw_rate_maps[trial], smoothed_pos_maps[trial], raw_pos_maps[trial], max_rates[trial], mean_rates[trial] = make_rate_maps(spike_data = current_trial_spikes_filtered,
                                   pos_data = obj.pos_data[trial], 
                                   adaptive_smoothing = True,
                                   alpha = 200) # for adaptive smoothing


        
        # Calculate spatial information from UNSMOOTHED rate and pos maps
        bits_per_spike[trial], bits_per_sec[trial] = spatial_info(raw_rate_maps[trial], raw_pos_maps[trial])
        
    
        # Save rate maps & spatial information to dfs
        for cluster in rate_maps[trial].keys():

            spatial_info_all.loc[f'{trial_name}_{cluster}', ['bits_per_spike', 'bits_per_sec', 'age']] = [bits_per_spike[trial][cluster], bits_per_sec[trial][cluster], obj.age]
            rates_all.loc[f'{trial_name}_{cluster}', ['mean_rates', 'max_rates', 'age']] = [mean_rates[trial][cluster], max_rates[trial][cluster], obj.age]

            trial_type = f'{trial_name.split("_")[-2]}_{trial_name.split("_")[-1]}'
            rate_maps_all.at[f'{trial_name.split("_")[0]}_{trial_name.split("_")[1]}_{cluster}', trial_type] = rate_maps[trial][cluster]
            rate_maps_all.at[f'{trial_name.split("_")[0]}_{trial_name.split("_")[1]}_{cluster}', 'age'] = obj.age

        occupancy_maps_all.at[f'{trial_name.split("_")[0]}_{trial_name.split("_")[1]}', trial_type] = smoothed_pos_maps[trial]
        occupancy_maps_all.at[f'{trial_name.split("_")[0]}_{trial_name.split("_")[1]}', 'age'] = obj.age



    ## RATE MAP PLOTTING

    # Define the directory where you want to save the figures
    save_directory = f'/home/isabella/Documents/isabella/jake/ephys_analysis/figures/P{obj.age}_{session}'
    
    # Check if the directory exists, if not, create it
    if not os.path.exists(save_directory):
        os.makedirs(save_directory)
    
    # Plot rate maps for all clusters in session
    for cluster in obj.spike_data['cluster_info'].index:
        plot_cluster_across_sessions(rate_maps_dict = rate_maps,
                                     cluster_id = cluster,
                                     max_rates_dict =  max_rates,
                                     mean_rates_dict = mean_rates,
                                     spatial_info_dict = bits_per_spike,
                                     session = session,
                                     age = obj.age)
        
        plt.tight_layout()
        plt.savefig(f'{save_directory}/{cluster}_rate_maps.png')
        plt.show()
            
    # Save data to pickle
    rate_maps_all.to_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/rate_maps_all.pkl')
    occupancy_maps_all.to_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/occupancy_maps_all.pkl')
    spatial_info_all.to_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/spatial_info_all.pkl')
    rates_all.to_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/rates_all.pkl')
    


In [ ]:
# from scipy.io import savemat

# rate_maps_open_field_1 = rate_maps_all.to_dict()['open-field_1']
# rate_maps_open_field_2 = rate_maps_all.to_dict()['open-field_2']
# rate_maps_t_maze_1 = rate_maps_all.to_dict()['t-maze_1']
# rate_maps_t_maze_2 = rate_maps_all.to_dict()['t-maze_2']
# occupancy_maps_open_field_1 = occupancy_maps_all.to_dict()['open-field_1']
# occupancy_maps_open_field_2 = occupancy_maps_all.to_dict()['open-field_2']
# occupancy_maps_t_maze_1 = occupancy_maps_all.to_dict()['t-maze_1']
# occupancy_maps_t_maze_2 = occupancy_maps_all.to_dict()['t-maze_2']

# maps_list = [rate_maps_open_field_1, rate_maps_open_field_2, rate_maps_t_maze_1, rate_maps_t_maze_2,
#              occupancy_maps_open_field_1, occupancy_maps_open_field_2, occupancy_maps_t_maze_1, occupancy_maps_t_maze_2]

# # add letter to each value in dict keys to make them valid matlab field names
# for m in maps_list[0:4]:
#     keys = list(m.keys())
#     for k in keys:
#         m['a' + k] = m.pop(k)
# for m in maps_list[4:8]:
#     keys = list(m.keys())
#     for k in keys:
#         m['b' + k] = m.pop(k)

# savemat('rate_maps_open-field_1.mat', maps_list[0])
# savemat('rate_maps_open-field_2.mat', maps_list[1])
# savemat('rate_maps_t-maze_1.mat', maps_list[2])
# savemat('rate_maps_t-maze_2.mat', maps_list[3])
# savemat('occupancy_maps_open-field_1.mat', maps_list[4])
# savemat('occupancy_maps_open-field_2.mat', maps_list[5])
# savemat('occupancy_maps_t-maze_1.mat', maps_list[6])
# savemat('occupancy_maps_t-maze_2.mat', maps_list[7])

## Spatial Information & Stats

In [ ]:
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Set age bins
age_bins = [(22, 26), (27, 29), (30, 35)]

# Load pickled rate map data
rate_maps_all = pd.read_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/rate_maps_all.pkl')
spatial_info_df = pd.read_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/spatial_info_all.pkl')

# Format spatial info dataframe
spatial_info_df.dropna(inplace = True)
spatial_info_df.reset_index(inplace = True)
spatial_info_df = spatial_info_df.rename(columns = {'index': 'cell_id'})
spatial_info_df['trial_type'] = spatial_info_df['cell_id'].apply(lambda x: x.split('_')[-3])
spatial_info_df['age'] = spatial_info_df['age'].astype(int)

# Group by age bin
spatial_info_df['age_bin'] = pd.cut(spatial_info_df['age'], bins = [22, 26, 29, 35], labels = ['22-26', '27-29', '30-35'])
# Convert 'age_bin' column to categorical data type
spatial_info_df['age_bin'] = pd.Categorical(spatial_info_df['age_bin'], categories=['22-26', '27-29', '30-35'], ordered=True)

# Calculate mean and std of bits_per_spike and bits_per_sec
spatial_info_df_stats = spatial_info_df.groupby('age_bin').agg({'bits_per_spike': ['mean', 'std'], 'bits_per_sec': ['mean', 'std']}).reset_index()

# Perform ANOVA on age bin and bits per spike
anova_bits_per_spike = stats.f_oneway(spatial_info_df[spatial_info_df['age_bin'] == '22-26']['bits_per_spike'],
                       spatial_info_df[spatial_info_df['age_bin'] == '27-29']['bits_per_spike'],
                       spatial_info_df[spatial_info_df['age_bin'] == '30-35']['bits_per_spike'])
print(f'ANOVA for bits per spike: {anova_bits_per_spike}')

# Perform ANOVA on age bin and bits per sec
anova_bits_per_sec = stats.f_oneway(spatial_info_df[spatial_info_df['age_bin'] == '22-26']['bits_per_sec'],
                       spatial_info_df[spatial_info_df['age_bin'] == '27-29']['bits_per_sec'],
                       spatial_info_df[spatial_info_df['age_bin'] == '30-35']['bits_per_sec'])
print(f'ANOVA for bits per sec: {anova_bits_per_sec}')

# Replace string age bins with integers
age_bin_mapping = {'22-26': 1, '27-29': 2, '30-35': 3}
spatial_info_df['age_bin'] = spatial_info_df['age_bin'].replace(age_bin_mapping)

if anova_bits_per_spike[1] < 0.05:
    # Perform Tukey HSD test for bits per spike
    tukey_bits_per_spike = pairwise_tukeyhsd(spatial_info_df['bits_per_spike'].astype(float), spatial_info_df['age_bin'])
    print(f'Tukey HSD for bits per spike: {tukey_bits_per_spike}')
else:
    print('ANOVA for bits per spike not significant, skipping Tukey HSD test')

# if p < 0.05, perform Tukey HSD test
if anova_bits_per_sec[1] < 0.05:
    # Perform Tukey HSD test for bits per sec
    tukey_bits_per_sec = pairwise_tukeyhsd(spatial_info_df['bits_per_sec'].astype(float), spatial_info_df['age_bin'])
    print(f'Tukey HSD for bits per sec: {tukey_bits_per_sec}')
else:
    print('ANOVA for bits per sec not significant, skipping Tukey HSD test')

# Plot bits_per_spike and bits_per_sec for each age bin
fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (10, 5))
axes[0].errorbar(x = spatial_info_df_stats['age_bin'], y = spatial_info_df_stats['bits_per_spike']['mean'], yerr = spatial_info_df_stats['bits_per_spike']['std'], fmt = 'o')
axes[0].set_title('Bits per spike')
axes[0].set_xlabel('Age bin')
axes[0].set_ylabel('Bits per spike')
axes[0].set_ylim(0, 1.5)

axes[1].errorbar(x = spatial_info_df_stats['age_bin'], y = spatial_info_df_stats['bits_per_sec']['mean'], yerr = spatial_info_df_stats['bits_per_sec']['std'], fmt = 'o')
axes[1].set_title('Bits per second')
axes[1].set_xlabel('Age bin')
axes[1].set_ylabel('Bits per second')
axes[1].set_ylim(0, 3)


## Spatial Correlation & Stats

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from spatial_correlation import spatial_correlation
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd

age_bins = [(22, 26), (27, 29), (30, 35)]

# Load pickled rate map data
rate_maps_all = pd.read_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/rate_maps_all.pkl')


# Calculate spatial correlation for each cluster across both trial types
for trial_type in ['open-field', 't-maze']:
    rate_maps_correlation = rate_maps_all[[f'{trial_type}_1', f'{trial_type}_2']].dropna()

    spatial_correlations = spatial_correlation(rate_maps_correlation[f'{trial_type}_1'].to_list(), rate_maps_correlation[f'{trial_type}_2'].to_list())

    rate_maps_all.loc[rate_maps_correlation.index, f'spatial_correlation_{trial_type}'] = spatial_correlations

rate_maps_all

# Keep only age and spatial correlation columns
spatial_correlations = rate_maps_all[['age', 'spatial_correlation_open-field', 'spatial_correlation_t-maze']].dropna().copy()
# Convert age to int
spatial_correlations['age'] = spatial_correlations['age'].astype(int)

# Group by age bin
spatial_correlations['age_bin'] = pd.cut(spatial_correlations['age'], bins = [22, 26, 29, 35], labels = ['22-26', '27-29', '30-35'])
# Convert 'age_bin' column to categorical data type
spatial_correlations['age_bin'] = pd.Categorical(spatial_correlations['age_bin'], categories=['22-26', '27-29', '30-35'], ordered=True)

# Calculate mean and std of spatial info correlations
spatial_correlations_stats = spatial_correlations.groupby('age_bin').agg({'spatial_correlation_open-field': ['mean', 'std'], 'spatial_correlation_t-maze': ['mean', 'std']}).reset_index()

# Perform ANOVA on age bin and open field spatial correlation
anova_open_field = stats.f_oneway(spatial_correlations[spatial_correlations['age_bin'] == '22-26']['spatial_correlation_open-field'],
                       spatial_correlations[spatial_correlations['age_bin'] == '27-29']['spatial_correlation_open-field'],
                       spatial_correlations[spatial_correlations['age_bin'] == '30-35']['spatial_correlation_open-field'])
print(f'ANOVA for spatial_correlation_open-field: {anova_open_field}')

# Perform ANOVA on age bin and t-maze spatial correlation
anova_tmaze = stats.f_oneway(spatial_correlations[spatial_correlations['age_bin'] == '22-26']['spatial_correlation_t-maze'],
                       spatial_correlations[spatial_correlations['age_bin'] == '27-29']['spatial_correlation_t-maze'],
                       spatial_correlations[spatial_correlations['age_bin'] == '30-35']['spatial_correlation_t-maze'])
print(f'ANOVA for spatial_correlation_t-maze: {anova_tmaze}')

# Replace string age bins with integers
age_bin_mapping = {'22-26': 1, '27-29': 2, '30-35': 3}
spatial_correlations['age_bin'] = spatial_correlations['age_bin'].replace(age_bin_mapping)

# if p < 0.05, perform Tukey HSD test
if anova_open_field[1] < 0.05:
    print('ANOVA significant, performing Tukey HSD test')
    # Perform Tukey HSD test for open field spatial correlation
    tukey_open_field = pairwise_tukeyhsd(spatial_correlations['spatial_correlation_open-field'], spatial_correlations['age_bin'])
    print(f'Tukey HSD for spatial_correlation_open-field: {tukey_open_field}')
else:
    print('ANOVA for open field not significant, skipping Tukey HSD test')

if anova_tmaze[1] < 0.05:
    # Perform Tukey HSD test for t-maze spatial correlation
    tukey_tmaze = pairwise_tukeyhsd(spatial_correlations['spatial_correlation_t-maze'], spatial_correlations['age_bin'])
    print(f'Tukey HSD for bits spatial_correlation_t-maze: {tukey_tmaze}')
else:
    print('ANOVA for T-maze not significant, skipping Tukey HSD test')

# Plot spatial correlation for open field and t-maze for each age bin
fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (10, 5))
axes[0].errorbar(x = spatial_correlations_stats['age_bin'], y = spatial_correlations_stats['spatial_correlation_open-field']['mean'], yerr = spatial_correlations_stats['spatial_correlation_open-field']['std'], fmt = 'o')
axes[0].set_title('Spatial Correlation Open Field')
axes[0].set_xlabel('Age bin')
axes[0].set_ylim(0, 1)
axes[0].set_ylabel('Spatial Correlation')

axes[1].errorbar(x = spatial_correlations_stats['age_bin'], y = spatial_correlations_stats['spatial_correlation_t-maze']['mean'], yerr = spatial_correlations_stats['spatial_correlation_t-maze']['std'], fmt = 'o')
axes[1].set_title('Spatial Correlation T-Maze')
axes[1].set_xlabel('Age bin')
axes[1].set_ylabel('Spatial Correlation')
axes[1].set_ylim(0, 1)

## Plot autocorrelograms

In [ ]:
# Add spatial info to mean_rates_all
mean_rates_all = pd.DataFrame.from_dict(rates_all, orient='index')
mean_rates_all = mean_rates_all.reset_index()

mean_rates_all['bits_per_sec'] = mean_rates_all['index'].apply(lambda x: spatial_info_all[x]['bits_per_sec'])
mean_rates_all['bits_per_spike'] = mean_rates_all['index'].apply(lambda x: spatial_info_all[x]['bits_per_spike'])


# Plot mean rate against bits_per_sec
fig, ax = plt.subplots(figsize = (10, 10))
ax.scatter(x = mean_rates_all['mean_rates'], y = mean_rates_all['bits_per_sec'])
ax.set_xlabel('Mean rate (Hz)')
ax.set_ylabel('Bits per second')

In [ ]:
import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from postprocessing.burst_index_and_autocorrelograms import compute_autocorrelograms_and_first_moment, plot_autocorrelogram

# Load pickled data
df_all_cells = pd.read_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/df_all_cells.pkl')
# Drop rows with no included clusters
df_all_cells = df_all_cells.dropna()
print(f'{len(df_all_cells.index)} sessions retained')

# Loop through all sessions and make all plots per cluster
for session, obj in df_all_cells['ephys_object'].items():
    
    save_directory = f'/home/isabella/Documents/isabella/jake/ephys_analysis/figures/P{obj.age}_{session}'
    
    # Load spikes for all good clusters
    
    autocorrelograms, first_moments = compute_autocorrelograms_and_first_moment(
        spike_times = obj.spike_data['spike_times'],
        spike_clusters = obj.spike_data['spike_clusters'],
        bin_size = 0.001, #1ms
        time_window = 0.05) #50ms
        # burst_threshold = 0.01) #10 ms
    
    # Loop through each good cluster and plot autocorrelogram, labelled with burst index
    for cluster, autocorrelogram in autocorrelograms.items():
        fig, ax = plot_autocorrelogram(session, cluster, autocorrelograms[cluster], first_moments[cluster]) # burst_indices[cluster]
        plt.savefig(f'{save_directory}/{cluster}_autocorrelogram.png')
        plt.show()
        

## Plot mean waveform

In [ ]:
import sys
import matplotlib.pyplot as plt
from phylib.io.model import load_model
import pandas as pd
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from ephys import *

# Load pickled data
df_all_cells = pd.read_pickle('/home/isabella/Documents/isabella/jake/ephys_analysis/processed_data/df_all_cells.pkl')
# Drop rows with no included clusters
df_all_cells = df_all_cells.dropna()
print(f'{len(df_all_cells.index)} sessions retained')

data_path = '/home/isabella/Documents/isabella/jake/recording_data'

# Loop through recording sessions
for n, session in enumerate(df_all_cells.index):
    
    # Get path to params.py
    obj = df_all_cells.loc[session, 'ephys_object']
    
    # Directory to save
    save_directory = f'/home/isabella/Documents/isabella/jake/ephys_analysis/figures/P{obj.age}_{session}'
    
    # Load mean waveforms for all clusters
    clusters = df_all_cells.loc[session, 'clusters_inc']
    mean_waveforms = obj.load_mean_waveforms(clusters)
    
    for cluster in clusters:
        
        mean_waveform = mean_waveforms[cluster]
        axes.plot(mean_waveform, c='black', alpha=1)
        axes.set_title(f"Session {session} (P{obj.age}) Cluster {cluster} Mean Waveform")
        axes.set_xlabel('Time (ms)')
        axes.set_ylabel('Voltage (uV)')
        # if np.min(mean_waveforms) > -200 and np.max(mean_waveforms) < 50:
        #     axes.set_ylim(-200, 50)

        ephys_sampling_rate = obj.spike_data['sampling_rate']
        
        # Calculate the time in milliseconds for each sample
        time_ms = np.arange(len(mean_waveform)) / ephys_sampling_rate * 1000

        # Set the x ticks to the calculated time in milliseconds
        axes.set_xticks(np.arange(0, n_samples, int(0.5 * ephys_sampling_rate / 1000)))
        axes.set_xticklabels(np.round(time_ms[::int(0.5 * ephys_sampling_rate / 1000)], 2))

        plt.tight_layout()
        plt.savefig(f'{save_directory}/{cluster}_waveform.png')
        plt.show()



## Combine all plots for a single cluster into one image for checking

In [ ]:
import os
from PIL import Image

def combine_images_vertically(image_list):
    """
    Combine a list of images vertically.
    
    Parameters:
        image_list (list): List of Image objects to combine.
        
    Returns:
        Image: Combined image.
    """
    images = [Image.open(i) for i in image_list]
    widths, heights = zip(*(i.size for i in images))
    
    total_height = sum(heights)
    max_width = max(widths)
    
    new_img = Image.new('RGB', (max_width, total_height))
    
    y_offset = 0
    x_offset = 700
    for n, img in enumerate(images):
        if 'waveform' in image_list[n]:
            new_img.paste(img, (x_offset, 0))
        else:
            new_img.paste(img, (0, y_offset))
            y_offset += img.height
    
    return new_img

def clear_combined_images(subdir):
    """
    Remove previously combined images in a given subdirectory.
    
    Parameters:
        subdir (str): The path to the subdirectory.
    """
    for file in os.listdir(subdir):
        if file.endswith('_combined.png'):
            os.remove(os.path.join(subdir, file))

def process_directory(main_directory):
    """
    Process the main directory to combine .png files with the same prefix within each subfolder.
    
    Parameters:
        main_directory (str): The path to the main directory containing subfolders.
    """  
    for subdir, _, files in os.walk(main_directory):
        clear_combined_images(subdir)
        
        prefix_to_files = {}
        
        for file in files:
            if file.endswith('.png') and not file.endswith('_combined.png'):
                prefix = f'{subdir}_{file[:3]}'
                if prefix not in prefix_to_files:
                    prefix_to_files[prefix] = []
                prefix_to_files[prefix].append(os.path.join(subdir, file))
                
        for prefix, file_paths in prefix_to_files.items():
            if len(file_paths) > 1:
                file_paths.sort()
                combined_img = combine_images_vertically(file_paths)
                combined_img_path = os.path.join(subdir, f"{prefix}_combined.png")
                combined_img.save(combined_img_path)
                

# Process all figures
process_directory('/home/isabella/Documents/isabella/jake/ephys_analysis/figures')


## Copy all combined images to a new directory

In [ ]:
import os
import shutil

def copy_png_files(source_directory, target_directory, substring):
    """
    Traverse through a directory and its subdirectories to find all .png files containing a specific substring.
    Then copy these files to a target directory.

    Parameters:
    - source_directory (str): The directory path to start the traversal.
    - target_directory (str): The directory where the files will be copied to.
    - substring (str): The substring that the .png file names should contain.
    """

    # Ensure that the target directory exists; if not, create it.
    if not os.path.exists(target_directory):
        os.makedirs(target_directory)

    # Traverse through the source directory and its subdirectories.
    for dirpath, _, filenames in os.walk(source_directory):
        for filename in filenames:
            # Check if the file is a .png and contains the specific substring.
            if filename.endswith('.png') and substring in filename:
                source_file_path = os.path.join(dirpath, filename)
                target_file_path = os.path.join(target_directory, filename)

                # Copy the file to the target directory.
                try:
                    shutil.copy2(source_file_path, target_file_path)
                except shutil.SameFileError:
                    print(f"File {filename} already exists in {target_directory}")

                # print(f"Copied {filename} to {target_directory}")

copy_png_files(source_directory = '/home/isabella/Documents/isabella/jake/ephys_analysis/figures', target_directory = '/home/isabella/Documents/isabella/jake/ephys_analysis/figures/all_cells_combined_figures', substring = 'combined.png')

## Export names of all cell combined images to csv for manual curation

In [ ]:
import os
import csv

# Specify the path to the directory containing the files
folder_path = '/home/isabella/Documents/isabella/jake/ephys_analysis/figures/all_cells_combined_figures'

# List all files and directories in the specified folder
all_files_and_dirs = os.listdir(folder_path)

# Filter out directories, keeping only files
only_files = [f for f in all_files_and_dirs if os.path.isfile(os.path.join(folder_path, f))]

# Remove file extensions from file names
file_names_without_extension = [os.path.splitext(f)[0] for f in only_files]

# Specify the path to the output CSV file
csv_file_path = '/home/isabella/Documents/isabella/jake/ephys_analysis/figures/all_cells_combined_figures/list_of_figures.csv'

# Write the list of file names to a CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    for file_name in file_names_without_extension:
        csv_writer.writerow([file_name[:-9]])


## Test